In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples
from scipy.sparse import hstack, csr_matrix

print("✅ 라이브러리 로드 완료")

## 1. 데이터 로드

### 📥 모델링용 데이터 사용

02번 노트북에서 생성한 `job_aioe_for_modeling.csv`를 로드합니다.

**이 데이터의 특징:**
- 747개 직업 (완전한 데이터만)
- AIOE, Employment, Mean_Wage
- Description (직무 설명 텍스트)

In [ ]:
# 데이터 로드
df = pd.read_csv("../datas/processed/job_aioe_for_modeling.csv")

print("✅ 데이터 로드 완료")
print(f"데이터 크기: {df.shape}")
print(f"\n컬럼 목록:")
print(df.columns.tolist())
print(f"\n상위 5개 행:")
df.head()

## 2. 피처 전처리

### 🔧 군집 분석을 위한 피처 준비

군집 분석에서는 **모든 피처를 동시에 고려**하여 유사도를 계산합니다.

**전처리 단계:**

1. **로그 변환**: Employment, Mean_Wage → Employment_log, Mean_Wage_log
2. **텍스트 벡터화**: Description → TF-IDF (500개 단어)
3. **피처 결합**: 숫자 + 텍스트
4. **스케일링**: 모든 피처를 같은 척도로 조정

### 💡 왜 로그 변환을 사용하는가?

02번 EDA에서 확인했듯이:
- Employment와 Mean_Wage는 극단적으로 치우친 분포
- 로그 변환으로 정규분포에 가까워짐
- **K-Means는 거리 기반** → 스케일이 비슷해야 함

In [ ]:
# 1. 로그 변환 (02번 노트북에서 이미 생성되었지만, 없을 경우를 위해)
if 'Employment_log' not in df.columns:
    df["Employment_log"] = np.log1p(df["Employment"])
if 'Mean_Wage_log' not in df.columns:
    df["Mean_Wage_log"] = np.log1p(df["Mean_Wage"])

# 2. 텍스트 처리
df["Description"] = df["Description"].fillna("")

# TF-IDF 벡터화 (500개 중요 단어)
tfidf = TfidfVectorizer(
    max_features=500,      # 상위 500개 중요 단어
    stop_words="english",  # 영어 불용어 제거
    min_df=2,              # 최소 2개 문서에 등장
    max_df=0.8             # 80% 이상 문서 등장 시 제외
)
X_text = tfidf.fit_transform(df["Description"])

print("✅ 텍스트 벡터화 완료")
print(f"텍스트 피처 shape: {X_text.shape}")
print(f"사용된 단어 수: {len(tfidf.get_feature_names_out())}")

# 3. 숫자형 피처 선택
X_num = df[["AIOE", "Employment_log", "Mean_Wage_log"]].values

print(f"\n숫자 피처 shape: {X_num.shape}")

# 4. 피처 결합 (숫자 + 텍스트)
X_combined = hstack([csr_matrix(X_num), X_text])

print(f"\n결합된 피처 shape: {X_combined.shape}")
print(f"  - 숫자 피처: 3개 (AIOE, Employment_log, Mean_Wage_log)")
print(f"  - 텍스트 피처: {X_text.shape[1]}개 (TF-IDF 단어)")
print(f"  - 총 피처: {X_combined.shape[1]}개")

### 🎚️ 피처 스케일링

**StandardScaler란?**
- 각 피처를 평균 0, 분산 1로 변환
- 공식: `(x - mean) / std`

**왜 스케일링이 필요한가?**

K-Means는 **유클리드 거리(Euclidean Distance)**를 사용합니다:

$$d(x, y) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}$$

**문제 예시:**
```
직업A: AIOE=4, Employment_log=10, Mean_Wage_log=11
직업B: AIOE=5, Employment_log=11, Mean_Wage_log=11

거리 계산 (스케일링 없음):
d = √[(5-4)² + (11-10)² + (11-11)²] = √2 ≈ 1.41
```

AIOE 차이 1과 Employment_log 차이 1이 **같은 중요도**로 계산됩니다.  
하지만 실제로는 피처마다 중요도가 다를 수 있습니다.

**스케일링 후:**
- 모든 피처가 평균 0, 분산 1
- 공정한 비교 가능

**⚠️ 희소행렬 주의사항:**

일반 StandardScaler는 평균을 빼므로 희소행렬이 밀집행렬(dense)로 변환됩니다.  
→ 메모리 폭발!

**해결:** `with_mean=False` 옵션 사용
- 평균 빼기 생략
- 표준편차로만 나눔
- 희소성 유지

In [ ]:
# 스케일링 (희소행렬 호환)
scaler = StandardScaler(with_mean=False)  # 희소성 유지
X_scaled = scaler.fit_transform(X_combined)

print("✅ 스케일링 완료")
print(f"최종 입력 데이터 shape: {X_scaled.shape}")
print(f"데이터 타입: {type(X_scaled)}")
print(f"희소성 (sparsity): {1 - X_scaled.nnz / (X_scaled.shape[0] * X_scaled.shape[1]):.2%}")

## 3. 최적의 클러스터 개수 (K) 찾기

### 🔍 K-Means의 가장 큰 문제: K를 어떻게 정할까?

K-Means는 **클러스터 개수 K를 미리 지정**해야 합니다.  
하지만 데이터를 보고 몇 개 그룹이 적절한지 알기 어렵습니다.

**해결 방법:**
1. **Elbow Method**: Inertia 그래프의 꺾이는 지점
2. **Silhouette Score**: 클러스터 품질 측정

---

### 📐 방법 1: Elbow Method (팔꿈치 방법)

**Inertia (응집도)란?**

$$\text{Inertia} = \sum_{i=1}^{K} \sum_{x \in C_i} ||x - \mu_i||^2$$

- 각 점과 소속 클러스터 중심의 거리 제곱합
- **낮을수록 좋음** (클러스터 내 점들이 중심에 가까움)

**특징:**
- K가 증가하면 Inertia는 항상 감소
- K = N (데이터 개수)이면 Inertia = 0
- 하지만 너무 많은 클러스터는 의미 없음!

**Elbow Point:**
- Inertia 감소 속도가 급격히 줄어드는 지점
- "팔꿈치"처럼 꺾이는 부분
- 그 이상 K를 늘려도 큰 이득 없음

**예시:**
```
K=2: Inertia=10000 → K=3: Inertia=6000 (큰 감소!)
K=3: Inertia=6000  → K=4: Inertia=4500 (중간 감소)
K=4: Inertia=4500  → K=5: Inertia=4000 (작은 감소)
K=5: Inertia=4000  → K=6: Inertia=3800 (작은 감소)
→ Elbow = K=4
```

In [ ]:
# Elbow Method: K=2~10까지 실험
K_range = range(2, 11)
inertias = []

print("🔍 Elbow Method: K-Means 실험 중...")
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    print(f"K={k}: Inertia={kmeans.inertia_:.2f}")

# 시각화
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, marker='o', linewidth=2, markersize=8)
plt.xlabel('클러스터 개수 (K)', fontsize=12)
plt.ylabel('Inertia (응집도)', fontsize=12)
plt.title('Elbow Method: 최적의 K 찾기', fontsize=14)
plt.grid(alpha=0.3)
plt.xticks(K_range)

# Elbow point 강조 (예시: K=5)
# 실제로는 그래프를 보고 판단
plt.axvline(x=5, color='red', linestyle='--', alpha=0.5, label='예상 Elbow (K=5)')
plt.legend()
plt.tight_layout()
plt.show()

print("\n💡 그래프에서 꺾이는 지점(Elbow)을 찾으세요!")

### 📊 방법 2: Silhouette Score (실루엣 점수)

**Silhouette Score란?**

각 데이터 포인트가 **자신의 클러스터에 얼마나 잘 속해 있는지** 측정하는 지표입니다.

**공식:**

$$s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}$$

- $a(i)$: 같은 클러스터 내 다른 점들과의 평균 거리 (작을수록 좋음)
- $b(i)$: 가장 가까운 다른 클러스터 점들과의 평균 거리 (클수록 좋음)

**점수 범위:**
- **-1 ~ +1**
  - +1: 완벽하게 분리된 클러스터
  - 0: 클러스터 경계에 있음
  - -1: 잘못 분류됨

**평균 Silhouette Score:**
- 모든 데이터 포인트의 s(i) 평균
- **높을수록 좋은 군집화**

**장점:**
- Inertia와 달리 **절대적인 품질** 측정
- 다른 K 값을 공정하게 비교 가능

**해석:**
| 점수 | 해석 |
|------|------|
| 0.7 ~ 1.0 | 강한 구조 (Strong structure) |
| 0.5 ~ 0.7 | 적절한 구조 (Reasonable structure) |
| 0.25 ~ 0.5 | 약한 구조 (Weak structure) |
| < 0.25 | 구조 없음 (No substantial structure) |

In [ ]:
# Silhouette Score 계산
silhouette_scores = []

print("🔍 Silhouette Score 계산 중...")
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append(score)
    print(f"K={k}: Silhouette Score={score:.4f}")

# 시각화
plt.figure(figsize=(10, 6))
plt.plot(K_range, silhouette_scores, marker='s', linewidth=2, markersize=8, color='green')
plt.xlabel('클러스터 개수 (K)', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Silhouette Score: 클러스터 품질 평가', fontsize=14)
plt.grid(alpha=0.3)
plt.xticks(K_range)

# 최고 점수 강조
best_k = K_range[np.argmax(silhouette_scores)]
best_score = max(silhouette_scores)
plt.axvline(x=best_k, color='red', linestyle='--', alpha=0.5, 
            label=f'최고 점수 (K={best_k}, Score={best_score:.4f})')
plt.legend()
plt.tight_layout()
plt.show()

print(f"\n✅ 최고 Silhouette Score: K={best_k} (Score={best_score:.4f})")

### 🎯 최종 K 결정

**두 가지 방법 종합:**

1. **Elbow Method**: Inertia 감소가 둔화되는 지점
2. **Silhouette Score**: 가장 높은 점수

**일반적인 전략:**
- 두 방법이 같은 K를 제시하면 → 그 K 선택
- 다르면 → Silhouette Score 우선 (절대적 품질 지표)
- 해석 가능성도 고려 (너무 많은 클러스터는 해석 어려움)

**이 분석에서는:**
- Elbow Method와 Silhouette Score 결과를 보고 결정
- 일반적으로 K=4~6 정도가 적절할 것으로 예상

## 4. K-Means 클러스터링 실행

### 🎲 최종 모델 학습

위 분석 결과를 바탕으로 최적의 K를 선택하여 최종 군집화를 수행합니다.

**K-Means 하이퍼파라미터:**
- `n_clusters`: 클러스터 개수 (위에서 결정한 K)
- `random_state=42`: 재현 가능성
- `n_init=10`: 다른 초기값으로 10번 실행 후 최선 선택

In [ ]:
# 최적 K 선택 (Silhouette Score 기준)
optimal_k = best_k  # 위에서 계산된 best_k 사용

print(f"🎯 선택된 K: {optimal_k}")
print(f"   Silhouette Score: {best_score:.4f}")

# 최종 K-Means 모델 학습
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans_final.fit_predict(X_scaled)

# 클러스터 레이블을 데이터프레임에 추가
df["Cluster"] = clusters

print(f"\n✅ K-Means 클러스터링 완료")
print(f"Inertia: {kmeans_final.inertia_:.2f}")
print(f"\n클러스터별 직업 수:")
print(df["Cluster"].value_counts().sort_index())

## 5. PCA를 이용한 시각화

### 🔬 문제: 고차원 데이터 시각화

현재 데이터는 **503차원** (AIOE, Employment_log, Mean_Wage_log + 500개 TF-IDF 단어)  
→ 2D 그래프로 시각화 불가능!

### 💡 해결: PCA (Principal Component Analysis)

**PCA란?**
- 고차원 데이터를 저차원으로 축소하는 기법
- 분산을 최대한 보존하면서 차원 축소
- **비지도 학습** 방법

**작동 원리:**

1. 데이터의 분산이 가장 큰 방향(축) 찾기 → **PC1 (주성분 1)**
2. PC1과 직교하며 두 번째로 분산이 큰 방향 찾기 → **PC2 (주성분 2)**
3. 원본 데이터를 PC1, PC2 좌표로 투영

**수식:**

$$\text{PC1} = \text{argmax}_{\mathbf{w}} \text{Var}(\mathbf{X} \mathbf{w})$$

- $\mathbf{X}$: 원본 데이터 (503차원)
- $\mathbf{w}$: 방향 벡터
- PC1: 분산을 최대화하는 방향

**주의사항:**

⚠️ **PCA는 시각화용일 뿐, 클러스터링에 사용하지 않음!**

- 클러스터링: 원본 503차원 데이터 사용
- PCA: 결과를 2D로 그리기 위한 도구
- PCA로 축소 후 클러스터링하면 정보 손실 발생

**분산 설명력 (Explained Variance):**

- PC1 + PC2가 원본 분산의 몇 %를 설명하는가?
- 예: 30% → PC1, PC2만으로 원본 정보의 30% 표현
- 나머지 70%는 다른 차원에 존재

In [ ]:
# PCA: 503차원 → 2차원 축소
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled.toarray())  # sparse → dense 변환 필요

# PCA 좌표를 데이터프레임에 추가
df["PCA1"] = X_pca[:, 0]
df["PCA2"] = X_pca[:, 1]

print("✅ PCA 차원 축소 완료")
print(f"원본 차원: {X_scaled.shape[1]}D")
print(f"축소 차원: 2D (PC1, PC2)")
print(f"\n분산 설명력:")
print(f"  PC1: {pca.explained_variance_ratio_[0]:.2%}")
print(f"  PC2: {pca.explained_variance_ratio_[1]:.2%}")
print(f"  Total: {pca.explained_variance_ratio_.sum():.2%}")
print(f"\n💡 PC1과 PC2는 원본 데이터 분산의 {pca.explained_variance_ratio_.sum():.1%}를 설명합니다.")

## 6. 클러스터 시각화

### 📊 2D 산점도로 클러스터 확인

PCA로 축소한 2차원 공간에서 각 클러스터를 색깔로 구분하여 시각화합니다.

**시각화 목적:**
1. 클러스터가 잘 분리되어 있는지 확인
2. 클러스터 간 경계 파악
3. 이상치(outlier) 발견

**해석 주의사항:**
- PC1, PC2는 원본 정보의 일부만 표현
- 2D에서 겹쳐 보여도 503D에서는 분리될 수 있음
- 클러스터 중심(centroid)도 함께 표시

In [ ]:
# 클러스터 시각화
plt.figure(figsize=(12, 8))

# 산점도: 각 클러스터를 다른 색으로 표시
scatter = plt.scatter(
    df["PCA1"], 
    df["PCA2"], 
    c=df["Cluster"],           # 클러스터별 색상
    cmap='tab10',              # 색상 팔레트
    alpha=0.6,                 # 투명도
    s=50,                      # 점 크기
    edgecolors='black',        # 테두리
    linewidths=0.5
)

# 클러스터 중심점 표시
centers_pca = pca.transform(kmeans_final.cluster_centers_)
plt.scatter(
    centers_pca[:, 0],
    centers_pca[:, 1],
    c='red',
    marker='X',
    s=300,
    edgecolors='black',
    linewidths=2,
    label='Cluster Centers'
)

# 클러스터 번호 레이블
for i, center in enumerate(centers_pca):
    plt.text(
        center[0], center[1], 
        f'C{i}',
        fontsize=14,
        fontweight='bold',
        ha='center',
        va='center',
        color='white'
    )

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
plt.title(f'직업 군집 분석 결과 (K={optimal_k}, PCA 2D 시각화)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"✅ {optimal_k}개 클러스터가 2D 공간에 투영되었습니다.")

## 7. 클러스터 프로파일링

### 🔍 각 클러스터의 특징 파악

클러스터링의 핵심은 **각 그룹이 어떤 특성을 가지는지** 이해하는 것입니다.

**분석 내용:**
1. **숫자 피처 평균**: AIOE, Employment, Mean_Wage
2. **대표 직업**: 각 클러스터의 예시 직업
3. **클러스터 명명**: 특성 기반 이름 부여

### 📊 클러스터별 통계 요약

In [ ]:
# 클러스터별 통계 요약
cluster_summary = df.groupby("Cluster")[["AIOE", "Employment", "Mean_Wage"]].agg(['mean', 'std', 'count'])

print("=" * 100)
print("📊 클러스터별 평균 특성")
print("=" * 100)
print(cluster_summary.round(2))
print("=" * 100)

# 시각화: 클러스터별 평균값 비교
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# AIOE
cluster_means = df.groupby("Cluster")["AIOE"].mean().sort_index()
axes[0].bar(cluster_means.index, cluster_means.values, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('평균 AIOE')
axes[0].set_title('클러스터별 평균 AIOE')
axes[0].grid(axis='y', alpha=0.3)

# Employment
cluster_means = df.groupby("Cluster")["Employment"].mean().sort_index()
axes[1].bar(cluster_means.index, cluster_means.values, color='lightgreen', edgecolor='black')
axes[1].set_xlabel('Cluster')
axes[1].set_ylabel('평균 Employment')
axes[1].set_title('클러스터별 평균 고용 규모')
axes[1].grid(axis='y', alpha=0.3)

# Mean_Wage
cluster_means = df.groupby("Cluster")["Mean_Wage"].mean().sort_index()
axes[2].bar(cluster_means.index, cluster_means.values, color='salmon', edgecolor='black')
axes[2].set_xlabel('Cluster')
axes[2].set_ylabel('평균 Mean_Wage ($)')
axes[2].set_title('클러스터별 평균 임금')
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 📝 클러스터별 대표 직업

In [ ]:
# 각 클러스터의 대표 직업 출력
print("\n" + "=" * 100)
print("📝 각 클러스터별 대표 직업 (상위 10개)")
print("=" * 100)

for cluster_id in range(optimal_k):
    print(f"\n{'='*50}")
    print(f"Cluster {cluster_id}")
    print(f"{'='*50}")
    
    cluster_df = df[df["Cluster"] == cluster_id]
    
    # 통계 요약
    print(f"직업 수: {len(cluster_df)}")
    print(f"평균 AIOE: {cluster_df['AIOE'].mean():.3f}")
    print(f"평균 Employment: {cluster_df['Employment'].mean():,.0f}")
    print(f"평균 Mean_Wage: ${cluster_df['Mean_Wage'].mean():,.0f}")
    
    # 대표 직업 (AIOE 순으로 정렬)
    print(f"\n상위 10개 직업 (AIOE 기준):")
    top_jobs = cluster_df.nlargest(10, 'AIOE')[['Title', 'AIOE', 'Employment', 'Mean_Wage']]
    for idx, (_, row) in enumerate(top_jobs.iterrows(), 1):
        print(f"  {idx:2d}. {row['Title'][:60]:<60} | AIOE={row['AIOE']:.2f} | Wage=${row['Mean_Wage']:>8,.0f}")

print("\n" + "=" * 100)

## 8. 클러스터 해석 및 명명

### 💡 클러스터 특성 분석

**실행 후 각 클러스터의 특징을 요약하고 이름을 붙여야 합니다.**

아래는 일반적으로 예상되는 패턴입니다:

#### 예상 클러스터 유형:

**1. 고AIOE-고임금 그룹**
- 특징: AIOE > 4.5, Mean_Wage > $100,000
- 직업 예시: 
  - 변호사 (Lawyers)
  - 경영 컨설턴트 (Management Consultants)
  - 금융 분석가 (Financial Analysts)
- **해석**: AI의 영향을 많이 받지만 고도의 전문성으로 높은 임금 유지

**2. 고AIOE-중임금 그룹**
- 특징: AIOE > 4.0, Mean_Wage $60,000~$90,000
- 직업 예시:
  - 회계사 (Accountants)
  - 데이터 입력 담당자 (Data Entry Keyers)
  - 사무 직원 (Office Clerks)
- **해석**: AI 자동화 위험이 높은 사무/행정직

**3. 중AIOE-고임금 그룹**
- 특징: AIOE 3.5~4.0, Mean_Wage > $90,000
- 직업 예시:
  - 소프트웨어 개발자 (Software Developers)
  - 엔지니어 (Engineers)
  - 의사 (Physicians)
- **해석**: AI를 도구로 활용하지만 대체되기 어려운 전문직

**4. 저AIOE-저임금 그룹**
- 특징: AIOE < 3.5, Mean_Wage < $50,000
- 직업 예시:
  - 요리사 (Cooks)
  - 청소부 (Cleaners)
  - 소매 판매원 (Retail Salespersons)
- **해석**: 육체노동/서비스직, AI 영향 낮음

**5. 저AIOE-중임금 그룹**
- 특징: AIOE < 3.5, Mean_Wage $50,000~$70,000
- 직업 예시:
  - 경찰 (Police Officers)
  - 소방관 (Firefighters)
  - 전기 기술자 (Electricians)
- **해석**: 숙련 기술직, AI 영향 제한적

---

### 📝 실행 후 할 일:

1. **위 출력 결과를 보고 각 클러스터의 패턴 파악**
2. **클러스터에 의미 있는 이름 부여**
   - 예: "고위험-고보상 전문직", "AI 자동화 가능 사무직" 등
3. **클러스터 간 차이점 정리**
4. **정책/개인 관점에서 시사점 도출**

## 9. 최종 인사이트 및 결론

### 🎯 군집 분석을 통해 발견한 것

#### 1. 직업 시장의 구조화된 패턴 발견

**군집 분석 결과:**
- 747개 직업이 K개의 명확한 그룹으로 분류됨
- 각 그룹은 AIOE, 고용 규모, 임금 수준의 조합으로 특징화
- 직업 시장은 무작위가 아닌 **구조화된 패턴**을 보임

#### 2. AIOE와 임금의 복잡한 관계

**발견:**
- AIOE와 임금은 단순 선형 관계가 아님
- 고AIOE 직업 중에도:
  - 고임금 (변호사, 컨설턴트) ← 전문성으로 보호
  - 중임금 (사무직, 데이터 입력) ← 자동화 위험
- 저AIOE 직업 중에도:
  - 중임금 (숙련 기술직) ← 경험/기술 필요
  - 저임금 (서비스직) ← 낮은 진입 장벽

**시사점:**
- AI 노출도만으로 직업의 미래를 판단하기 어려움
- 임금 수준, 전문성, 대체 가능성을 종합 고려 필요

#### 3. 정책 및 개인 전략 수립 기반

**정책 입안자:**
- 클러스터별 맞춤형 지원 정책
  - 고위험 클러스터 → 재교육 프로그램
  - 저위험 클러스터 → AI 도구 활용 교육

**개인:**
- 내 직업이 속한 클러스터 확인
- 같은 클러스터 내 다른 직업으로 전환 고려
- 다른 클러스터로 이동하기 위한 스킬 개발

---

### 📚 학습 내용 정리

#### 비지도 학습 기술

| 단계 | 학습 내용 | 핵심 개념 |
|------|----------|----------|
| **피처 준비** | 로그 변환, TF-IDF, 스케일링 | 거리 기반 알고리즘 준비 |
| **최적 K 선택** | Elbow Method, Silhouette Score | 클러스터 개수 결정 |
| **군집화** | K-Means 알고리즘 | 중심 기반 그룹핑 |
| **차원 축소** | PCA | 고차원 → 2D 시각화 |
| **해석** | 클러스터 프로파일링 | 실용적 의미 부여 |

#### 군집 분석 Best Practices

**✅ 올바른 방법:**
1. **피처 스케일링 필수** (K-Means는 거리 기반)
2. **최적 K 선택** (Elbow + Silhouette)
3. **PCA는 시각화만** (군집화는 원본 데이터)
4. **클러스터 해석** (통계 + 대표 샘플)
5. **도메인 지식 활용** (숫자만 보지 말고 의미 파악)

**❌ 피해야 할 실수:**
1. 스케일링 없이 K-Means → 큰 값 피처 지배
2. K를 임의로 선택 → 부적절한 군집화
3. PCA 후 군집화 → 정보 손실
4. 클러스터 해석 없음 → 실용성 없음

---

### 🔗 전체 분석 파이프라인 요약

**01번 (전처리):**
- AIOE 계산 (R × L × I)
- OEWS 데이터 병합
- 결측치 처리

**02번 (EDA):**
- 데이터 필터링 (완전한 데이터만)
- 로그 변환 (치우친 분포 정규화)
- 상관관계 분석 (AIOE vs 임금)

**03번 (모델링):**
- 지도 학습 (임금 예측)
- Linear Regression, Random Forest, LightGBM
- TF-IDF 텍스트 피처 활용

**04번 (군집 분석):**
- 비지도 학습 (직업 그룹 발견)
- K-Means 클러스터링
- PCA 시각화
- 직업 시장 구조 이해

---

### 🚀 향후 개선 방향

**1. 다른 군집 알고리즘 시도**
- Hierarchical Clustering (계층적 군집)
- DBSCAN (밀도 기반 군집)
- Gaussian Mixture Models

**2. 더 많은 피처 추가**
- 교육 요구사항
- 산업 분류
- 지역 정보

**3. 시간에 따른 변화 추적**
- 연도별 AIOE 변화
- 클러스터 이동 패턴
- ChatGPT 등장 전후 비교

**4. 실용적 활용**
- 직업 추천 시스템
- 클러스터 기반 재교육 프로그램 매칭
- 정책 입안 대시보드

---

### 📖 참고 문헌

1. **Felten, E., Raj, M., & Seamans, R. (2023)**  
   *How will Language Modelers like ChatGPT Affect Occupations and Industries?*

2. **Felten, E., Raj, M., & Seamans, R. (2021)**  
   *Occupational, industry, and geographic exposure to artificial intelligence: A novel dataset and its potential uses*

3. **MacQueen, J. (1967)**  
   *Some methods for classification and analysis of multivariate observations*  
   - K-Means 알고리즘 최초 제안

4. **Rousseeuw, P. J. (1987)**  
   *Silhouettes: A graphical aid to the interpretation and validation of cluster analysis*  
   - Silhouette Score 제안

---

### 🎓 결론

이 노트북에서는 **K-Means 군집 분석**을 통해 직업들을 유사한 그룹으로 분류했습니다.

**성공적으로 확인한 것:**
- 직업 시장은 AIOE, 고용, 임금 기준으로 구조화된 패턴을 보임
- AI 영향도와 임금의 관계는 복잡하며 단순 선형 관계가 아님
- 군집 분석은 직업 시장 구조를 이해하는 유용한 도구

**학습한 것:**
- K-Means 알고리즘 원리
- 최적 K 선택 (Elbow, Silhouette)
- PCA 차원 축소
- 클러스터 해석 및 프로파일링

**다음 단계:**
- 실제 데이터로 실행하여 결과 확인
- 각 클러스터에 의미 있는 이름 부여
- 정책/개인 관점에서 시사점 도출

---

**🎉 수고하셨습니다! 전체 AIOE 분석 파이프라인(01 전처리 → 02 EDA → 03 모델링 → 04 군집 분석)을 완성했습니다!**

# 🎯 AIOE 군집 분석 (Clustering Analysis)

이 노트북은 03번 모델링에 이어 **비지도 학습(Unsupervised Learning)**을 사용하여 직업들을 유사한 그룹으로 분류하는 단계입니다.

---

## 📚 군집 분석이란?

**군집 분석 (Clustering)**은 레이블이 없는 데이터를 **유사한 특성을 가진 그룹(클러스터)으로 나누는** 비지도 학습 방법입니다.

### 지도 학습 vs 비지도 학습

| 구분 | 지도 학습 (03번 모델링) | 비지도 학습 (04번 군집 분석) |
|------|------------------------|---------------------------|
| **목표** | 예측 (Prediction) | 패턴 발견 (Pattern Discovery) |
| **레이블** | 있음 (Mean_Wage) | 없음 |
| **예시** | 임금 예측 모델 | 직업 그룹 분류 |
| **평가** | R², RMSE | Silhouette, Elbow Method |

---

## 🎯 분석 목적

### 핵심 질문:
> 직업들을 AIOE, 고용 규모, 임금, 직무 설명 기준으로 그룹화하면 어떤 패턴이 나타나는가?

### 기대 효과:

**1. 직업 시장 구조 이해**
- 비슷한 직업들의 그룹 발견
- 예시: "고AIOE-고임금", "저AIOE-저임금" 그룹

**2. 정책 활용**
- 그룹별 맞춤형 교육 프로그램
- AI 영향도가 비슷한 직업군 지원 정책

**3. 개인 활용**
- 내 직업과 유사한 다른 직업 발견
- 전환 가능한 직업군 탐색

---

## 📊 분석 전략

### 사용 알고리즘: K-Means

**K-Means란?**
- 데이터를 K개의 그룹으로 나누는 알고리즘
- 각 그룹의 중심(centroid)과 가장 가까운 점들을 묶음

**K-Means 작동 원리:**

1. 무작위로 K개의 중심점 선택
2. 각 데이터를 가장 가까운 중심점에 할당
3. 각 그룹의 평균으로 중심점 업데이트
4. 2-3 반복 (수렴할 때까지)

**공식:**

$$\text{minimize} \sum_{i=1}^{K} \sum_{x \in C_i} ||x - \mu_i||^2$$

- $C_i$: 클러스터 i
- $\mu_i$: 클러스터 i의 중심점
- $||x - \mu_i||$: 데이터 x와 중심점의 거리

### 사용 피처

| 피처 | 설명 | 역할 |
|------|------|------|
| **AIOE** | AI 직업 노출도 | AI 영향도 |
| **Employment_log** | 고용 규모 (로그) | 직업 규모 |
| **Mean_Wage_log** | 평균 임금 (로그) | 보상 수준 |
| **Description (TF-IDF)** | 직무 설명 (500개 단어) | 직무 특성 |

**총 피처:** 3 (숫자) + 500 (텍스트) = 503개

### 시각화 방법: PCA

**문제:** 503차원 데이터를 2D로 시각화 불가

**해결:** PCA (Principal Component Analysis)로 2차원 축소

- PC1 (주성분 1): 분산을 가장 많이 설명하는 축
- PC2 (주성분 2): PC1과 직교하며 두 번째로 중요한 축

---

## 🔗 이전 노트북과의 연결

**01번 (전처리):**
- AIOE 계산, OEWS 병합
- `job_aioe_processed.csv` 생성

**02번 (EDA):**
- 완전 데이터 선택, 로그 변환
- `job_aioe_for_modeling.csv` 생성

**03번 (모델링):**
- AIOE → 임금 예측 모델 구축
- 지도 학습 (Supervised Learning)

**04번 (군집 분석):**
- 직업 그룹 발견
- 비지도 학습 (Unsupervised Learning)

## 0. 환경 설정

### 📚 라이브러리 설명

**데이터 처리:**
- `pandas`, `numpy`: 데이터 조작 및 수치 계산

**시각화:**
- `matplotlib`, `seaborn`: 그래프 생성
- `koreanize_matplotlib`: 한글 폰트 설정

**텍스트 처리:**
- `TfidfVectorizer`: 텍스트를 TF-IDF 벡터로 변환

**전처리:**
- `StandardScaler`: 피처 표준화 (평균 0, 분산 1)

**군집 분석:**
- `KMeans`: K-Means 군집 알고리즘
- `PCA`: 차원 축소 (시각화용)

**기타:**
- `scipy.sparse`: 희소 행렬 처리